# تشخیص محصولات باغی خراب از سالم با استفاده از SVM 
 در این notebook تلاش می‌کنیم تا با در ابتدا با دیتای عکسی و نحوه استفاده از آن برای طبقه‌بندی آشنا شویم و سپس با استفاده از الگوریتم SVM پیاده سازی شده در کتابخانه Sklearn برای تشخیص محصول سالم از غیر سلام استفاده می‌کنیم. نتایج این notebook در مقابل نتایج استفاده از  CNN بر روی همین داده قرار می‌گیرد.




1.  ابتدا داده را دانلود میکنیم و در پوشه‌ای به همان نام قرار می‌دهیم.


In [0]:
!git clone https://github.com/spMohanty/PlantVillage-Dataset.git
!mkdir input
!cp -r PlantVillage-Dataset/raw/color/* ./input
!rm -rf PlantVillage-Dataset

2.    برای پیش‌پردازش و طراحی شبکه کتابخانه‌های لازم را import می‌کنیم.


In [0]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split,KFold

from skimage.io import imread
from skimage.transform import resize



3.  تابع زیر تلاش می‌کند که یک عکس را از روی دیسک بخواند و آنرا تبدیل به یک آرایه بکند.

4.   در زیر تلاش می‌کنیم تا هر عکس را در دایرکتوری مربوط به خودش بخوانیم و عکس را با استفاده از تابع بالا به آرایه تبدیل می‌کنیم و از نام دایرکتوری‌ش به آن لیبلی می‌دهیم.


In [0]:
im_for_show = []
def  load_image_files(container_path, dimension=(64, 64,3)):
    
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    
    flat_data = []
    target = []
    for i, direc in enumerate(folders[:10]):
        print("[INFO] {}".format(direc))
        for file in direc.iterdir():
            try:
                img = imread(file)
                im_for_show.append(img)
                img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
                flat_data.append(img_resized.flatten()) 
                images.append(img_resized)
                target.append(i)
            except:
                pass
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [0]:
image_dataset = load_image_files("input")



5.   سپس داده را به دو بخش تست و ترین تقسیم می‌کنیم. 



In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3,random_state=100)

6.  با استفاده از تابع SearchGrid می‌توانیم حالت‌های مختلف برای هایپر پارامترهای الگوریتم SVM  را انتخاب کنیم.
7. با استفاده از KFolds از کراس ولیدیشن برای آماده سازی دیتا برای چند مرخله ترین و تست جدا استفاده می‌کنیم.

8.   حال با آماده بودن دیتا و الگوریتم، کار ترین را آغاز می‌کنیم.


In [0]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 10)
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid,cv = folds, verbose = 1, n_jobs=-1)
clf.fit(X_train, y_train)

In [0]:
y_pred = clf.predict(X_test)


9.  بعد از پیشبینی دقت را بررسی و مقایسه می‌کنیم.



In [0]:
print("Classification report for - \n{}:\n{}\n".format(
    clf, metrics.classification_report(y_test, y_pred)))

10.   تکه کد زیر برای نشان دادن عکس‌ها استفاده می‌شود


In [0]:
def plot_image_grid(images, nb_rows, nb_cols, figsize=(2, 2)):
    assert len(images) == nb_rows*nb_cols, "Number of images should be the same as (nb_rows*nb_cols)"
    fig, axs = plt.subplots(nb_rows, nb_cols, figsize=figsize)
    
    n = 0
    for i in range(0, nb_rows):
        for j in range(0, nb_cols):
            axs[i, j].axis('off')
            axs[i, j].imshow(images[n])
            n += 1        

In [0]:
print(label_list[0])
plot_image_grid(image_list_fs[0:100], 10, 10)